In [ ]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
# check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

X = tf.placeholder(tf.float32, [None, 28*28])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
lr = 0.01

# Layer1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev = 0.01))
#    Conv          ->(?, 28, 28, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
#    Pool          ->(?, 14, 14, 32)
L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides=[1,2,2,1], padding='SAME')

# Layer2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv          ->(?, 14, 14, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
#    Pool          ->(?, 7, 7, 64)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L2 = tf.reshape(L2, [-1, 7 * 7 * 64])

# Final Fully Connected(FC, Dense) Layer
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10], 
                     initializer = tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
hypo = tf.matmul(L2, W3) + b

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypo, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

correct = tf.equal(tf.argmax(hypo, 1), tf.argmax(Y, 1))
accu = tf.reduce_mean(tf.cast(correct, tf.float32))

training_epochs = 10
batch_size = 100

# Training
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        x_batch, y_batch = mnist.train.next_batch(batch_size)
        feed = {X: x_batch, Y: y_batch}
        _, l = sess.run([optimizer, loss], feed_dict = feed)
        avg_loss += l/total_batch
    print('Epoch:', '%02d' %(epoch+1), 'Loss:', '{:.8f}'.format(avg_loss))
    
print('Learning finished.')

# Testing
print('Accuracy:', sess.run(accu, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 01 Loss: 0.15249166
Epoch: 02 Loss: 0.05002002
Epoch: 03 Loss: 0.03822474
Epoch: 04 Loss: 0.03268396
Epoch: 05 Loss: 0.02644057
Epoch: 06 Loss: 0.02720382
Epoch: 07 Loss: 0.02354336
Epoch: 08 Loss: 0.02079113
Epoch: 09 Loss: 0.02350275
Epoch: 10 Loss: 0.02247494
Learning finished.
